# Lleva a cabo la Validación del Modelo Entrenado comparando contra los objetos indicados en los XMLs correspondientes
Se basa en propuesta de https://towardsdatascience.com/evaluating-performance-of-an-object-detection-model-137a349c517b

0) Preparar ambiente e instalar paquetes:

In [ ]:
# nota se debe indicar la versión 1 de TF para compatibilidad del código
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

In [ ]:
#@title baja e instala los parquetes de 'Object Detection' de Tensor Flow a utilizar en el disco temporal de Colab (demora un ratito)
!pip install tf_slim

%cd /content
!git clone --quiet https://github.com/tensorflow/models.git

!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib

!pip install -q pycocotools

%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'
model_dir = 'training/'

!python object_detection/builders/model_builder_test.py

os.environ['PYTHONPATH'] += ':/content/models/research/object_detection:/content/models/research/slim/object_detection'


1) Cargar librerías:

In [ ]:
import os
import os.path
import sys
import numpy as np
import pandas as pd

from IPython.display import Image, display
from PIL import Image as ImPIL

import tensorflow as tf
from PIL import ImageColor
from PIL import ImageDraw

import copy
import xml.etree.cElementTree as ET

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

print ("Librerías cargadas.")

2) Montar el Drive:

In [ ]:
# monta Google Drive:
# Nota: la primera vez se debe confirmar el uso logueandose en "Google Drive File Stream" y obteniendo código de autentificación.
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
# configuración de directorios local en Google Drive
drive_path = '/content/gdrive/My Drive/GEMIS/objDetectionCursogramas'
data_dir_path = drive_path + '/Cursogramas'

print("Configuración de archivos definida")

3) Cargar el modelo entrenado:

In [ ]:
#@title carga el modelo de object detection entrenado

# path donde está el modelo exportado
ModelObjDetEntrenado = drive_path + '/TF_model/frozen_inference_graph.pb'

# archivo con lista de etiquetas para mostrar 
labelMapFile = data_dir_path + '/label_map.pbtxt'

# se debe ubicar en el directorio correspondiente
%cd /content/models/research/object_detection

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

# This is needed to display the images.
%matplotlib inline

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
 
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(ModelObjDetEntrenado, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

label_map = label_map_util.load_labelmap(labelMapFile)
categories = label_map_util.convert_label_map_to_categories(
    label_map, max_num_classes=90, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

# función auxiliar para ejecutar el modelo
def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {
                output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                'num_detections', 'detection_boxes', 'detection_scores',
                'detection_classes', 'detection_masks'
            ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                        tensor_name)
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(
                    tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(
                    tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(
                    tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [
                                           real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [
                                           real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image.shape[0], image.shape[1])
                detection_masks_reframed = tf.cast(
                    tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(detection_masks_reframed, 0)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict,
                                   feed_dict={image_tensor: np.expand_dims(image, 0)})

            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict['detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict


print("Modelo objDetector-Carteles cargado: [", ModelObjDetEntrenado, "], [", labelMapFile, "] ")


4) Llevar a cabo la validación usando las imágenes y XML:

In [ ]:
#@title Define funciones auxiliares 

# función para cálculo de Intersection over Union (IoU) 
def calc_IoU( gt_bbox, pred_bbox):
    '''
    This function takes the predicted bounding box and ground truth bounding box and 
    return the IoU ratio
    '''
    x_topleft_gt, y_topleft_gt, x_bottomright_gt, y_bottomright_gt = gt_bbox
    x_topleft_p, y_topleft_p, x_bottomright_p, y_bottomright_p = pred_bbox
    
    if (x_topleft_gt > x_bottomright_gt) or (y_topleft_gt> y_bottomright_gt):
        raise AssertionError("Ground Truth Bounding Box is not correct")
    if (x_topleft_p > x_bottomright_p) or (y_topleft_p> y_bottomright_p):
        raise AssertionError("Predicted Bounding Box is not correct",x_topleft_p, x_bottomright_p,y_topleft_p,y_bottomright_gt)
        
         
    #if the GT bbox and predcited BBox do not overlap then iou=0
    if(x_bottomright_gt< x_topleft_p):
        # If bottom right of x-coordinate  GT  bbox is less than or above the top left of x coordinate of  the predicted BBox
        
        return 0.0
    if(y_bottomright_gt< y_topleft_p):  # If bottom right of y-coordinate  GT  bbox is less than or above the top left of y coordinate of  the predicted BBox
        
        return 0.0
    if(x_topleft_gt> x_bottomright_p): # If bottom right of x-coordinate  GT  bbox is greater than or below the bottom right  of x coordinate of  the predcited BBox
        
        return 0.0
    if(y_topleft_gt> y_bottomright_p): # If bottom right of y-coordinate  GT  bbox is greater than or below the bottom right  of y coordinate of  the predcited BBox
        
        return 0.0
    
    
    GT_bbox_area = (x_bottomright_gt -  x_topleft_gt + 1) * (  y_bottomright_gt -y_topleft_gt + 1)
    Pred_bbox_area =(x_bottomright_p - x_topleft_p + 1 ) * ( y_bottomright_p -y_topleft_p + 1)
    
    x_top_left =np.max([x_topleft_gt, x_topleft_p])
    y_top_left = np.max([y_topleft_gt, y_topleft_p])
    x_bottom_right = np.min([x_bottomright_gt, x_bottomright_p])
    y_bottom_right = np.min([y_bottomright_gt, y_bottomright_p])
    
    intersection_area = (x_bottom_right- x_top_left + 1) * (y_bottom_right-y_top_left  + 1)
    
    union_area = (GT_bbox_area + Pred_bbox_area - intersection_area)
   
    return intersection_area/union_area


# función para mostrar boxes en una imagen dada
def draw_box(draw, rangeObj, lineWidth, lineColor):  

    draw.line([(rangeObj[0], rangeObj[1]), 
                        (rangeObj[0], rangeObj[3]), 
                        (rangeObj[2], rangeObj[3]), 
                        (rangeObj[2], rangeObj[1]), 
                        (rangeObj[0], rangeObj[1])], 
                      width=lineWidth, fill=lineColor)

# función para mostrar boxes de una lista objetos en una imagen dada
def draw_boxes_listObj(draw, listObj, lineWidth, lineColor):
  
    for obj in listObj:
        draw_box(draw, obj[2], lineWidth, lineColor)

print("Funciones auxiliares definidas")   

In [ ]:
# define la carpeta donde están las imágenes para procesar

#dirTestImg = data_dir_path + '/validation/images' 
#dirTestXML = data_dir_path + '/validation/annotations' 

dirTestImg = data_dir_path + '/Generados' 
dirTestXML = data_dir_path + '/Generados' 

# levanta los XML de validación para dirTestXML
process_FileNames = [ fn for fn in os.listdir( dirTestXML ) if fn.endswith('.xml') ]
print("> Imágenes/XML a probar: ", len(process_FileNames))

In [ ]:
#@markdown ### Toma una muestra de las imágenes (si es necesario o se quiere) { run: "auto" }
porcMuestraImagenesProcesar = 5  #@param {type:"slider", min:0, max:100, step:5}
if porcMuestraImagenesProcesar>0 and porcMuestraImagenesProcesar<100:
  cantProcesar = int(len(process_FileNames)*porcMuestraImagenesProcesar/100)
  if cantProcesar==0:
    cantProcesar = 1
  process_FileNames = process_FileNames[:cantProcesar]
print("> Imágenes/XML a probar: ", len(process_FileNames))

In [ ]:
#@title define parámetros a utilizar { run: "auto" }

# define minima probabilidad a usar
minimaProbabilidadObjectosDetectados = 90 #@param {type:"slider", min:1, max:100, step:1.0}
minProbObjDet = minimaProbabilidadObjectosDetectados / 100.

# define si muestra detalle o no
muestraDetalleDebug = False  #@param {type:"boolean"}
muestraDetalleObjDetectadosEnImagen = False  #@param {type:"boolean"}
muestraDetalleComparacionEnImagen = "Solo con Error" #@param ["Ninguna", "Solo con Error", "Todas"]
muestraDetalleMetricasPorImagen = "Solo con Error" #@param ["Ninguna", "Solo con Error", "Todas"]
muestraDetalleMetricasPorClaseObjeto = True  #@param {type:"boolean"}

# define parámetro Intersection over Union (IoU) 
## si calc_IoU(r1, r2) ≥ coefIoU, se considera que se detectó el objecto correctamente, es Verdadero Positivo (VP)
## si calc_IoU(r1, r2) < coefIoU, se considera que se detectó el objecto con error, es Falso Positivo (FP)
## -> valor recomendado por defecto: 0,5 
## pero se usa menos para mejorar los resultados
coefIoU = 0.4 #@param {type:"slider", min:0.1, max:1, step:0.1}

In [ ]:
#@title Realizar la Validación del Modelo
# inicializa vector auxiliar para metricas y posiciones a usar
cantXMLProcesados = 0
metricasGral = [ 0, 0, 0, 0]
posVP = 0
posVN = 1
posFP = 2
posFN = 3

# inicializa vectores auxiliares para evaluación de objetos detectados
classObjModelo = []
classObjReal = []

# muestra parámetros
print("> Parámetros: ")
print("  minimaProbabilidadObjectosDetectados: ", minimaProbabilidadObjectosDetectados)
print("  coefIoU: ", coefIoU)
print("\n")
print("  muestraDetalleDebug: ", muestraDetalleDebug)
print("  muestraDetalleObjDetectadosEnImagen: ", muestraDetalleObjDetectadosEnImagen)
print("  muestraDetalleComparacionEnImagen: ", muestraDetalleComparacionEnImagen)
print("  muestraDetalleMetricasPorImagen: ", muestraDetalleMetricasPorImagen)
print("  muestraDetalleMetricasPorClaseObjeto: ", muestraDetalleMetricasPorClaseObjeto)
print("\n\n\n")

# Procesa los XMLs de las imágenes 
for xml_file in process_FileNames:

    # inicializa vectores auxiliares
    listObjsXML = []
    listObjsDetModelo = []
    
    print("\n------------------------------------------------------------------------------------------------------------")
    cantXMLProcesados = cantXMLProcesados + 1
    print("<", cantXMLProcesados, "> ", xml_file, ": ")

    # carga la info del XML original
    et = ET.parse(dirTestXML + '/' + xml_file)
    element = et.getroot()
    element_objs = element.findall('object') 
    element_filename = element.find('filename').text

    imagenProcesar = os.path.join(dirTestImg, element_filename)

    # carga los elementos en el archivo XML original para generar el nuevo
    for element_obj in element_objs:

        # obtiene la información actual de la imagen
        class_name = element_obj.find('name').text 

        # obtiene info del box actual
        obj_bbox = element_obj.find('bndbox')
        nuevoRangoIm = [ float(obj_bbox.find('xmin').text),
                        float(obj_bbox.find('ymin').text),
                        float(obj_bbox.find('xmax').text), 
                        float(obj_bbox.find('ymax').text) ]

        # calcula un valor para poder ordenar las figuras de arriba a abajo y izquierda a derecha
        centroideIm = nuevoRangoIm[1]*100000+nuevoRangoIm[0]

        # agrega a lista de objetos cargados del XML
        listObjsXML.append( (centroideIm, class_name, nuevoRangoIm) )

    # carga la imagen a procesar
    imageCargada = ImPIL.open(imagenProcesar) 

    # Convierte la imagen a escala de grises y luego a RGB 
    # (para sacarle los colores que tuviera previamente y dejarlo con 3 canales de profundidad)
    imageCargada = imageCargada.convert('L')
    imageCargada = imageCargada.convert('RGB')

    imCargada_ancho, imCargada_alto = imageCargada.size

    # prepara la imagen cargada
    image_np = load_image_into_numpy_array(imageCargada)
    image_np_expanded = np.expand_dims(image_np, axis=0)

    # ejecuta el modelo
    output_dict = run_inference_for_single_image(image_np, detection_graph)

    # procesa los objetos detectados
    for detClass, detBox, detScore in zip(  output_dict['detection_classes'], output_dict['detection_boxes'], output_dict['detection_scores'] ):

        # si el objeto detectado tiene un puntaje superior o igual al mínimo
        if detScore >= minProbObjDet:

              class_name = category_index[detClass]['name']

              # como las coordenadas están normalizadas las debe convertir 
              # teniendo en cuenta el tamaño de la imagen
              # además notar que vienen datas en otro orden
              # - detBox = (ini alto, ini ancho, fin alto, fin ancho)
              # - nuevoRangoIn = (ini ancho x1, ini alto y1, fin ancho x2, fin alto y2)    
              nuevoRangoIm = [detBox[1] * imCargada_ancho, 
                              detBox[0] * imCargada_alto,
                              detBox[3] * imCargada_ancho,
                              detBox[2] * imCargada_alto]

              # calcula un valor para poder ordenar las figuras de arriba a abajo y izquierda a derecha
              centroideIm = nuevoRangoIm[1]*100000+nuevoRangoIm[0]

              # agrega a lista de objetos detectados por el modelo
              listObjsDetModelo.append( (centroideIm, class_name, nuevoRangoIm) )          
    
    # ordena por el centroide las dos listas
    listObjsXML = sorted(listObjsXML, key=lambda objDet: objDet[0])  
    listObjsDetModelo = sorted(listObjsDetModelo, key=lambda objDet: objDet[0])  

    if muestraDetalleDebug:
        print("\n- Objetos del XML:")
        print(len(listObjsXML), " : ",listObjsXML)

        print("\n- Objetos detectados por el Modelo:")
        print(len(listObjsDetModelo), " : ", listObjsDetModelo)

    if muestraDetalleObjDetectadosEnImagen: 

        print("\n- Muestra los objetos del XML y Detectados en la Imagen:")
        # imagen auxiliar para mostrar recuadros de XML y modelo
        image_pil = copy.deepcopy(imageCargada.convert("RGB"))
        draw = ImageDraw.Draw(image_pil)
        im_width, im_height = image_pil.size    

        # genera los recuadros correspondientes del XML (en color verde)
        draw_boxes_listObj(draw, listObjsXML, 8, (0,255,0))
            
        # genera los recuadros correspondientes al Modelo (en color azul)
        draw_boxes_listObj(draw, listObjsDetModelo, 4, (0,0,255))

        imMostrar = ImPIL.fromarray(np.array(image_pil), 'RGB')
        display( imMostrar )
        print(" Nota colores:")
        print("    Objetos definidos en el XML, cuadros en VERDE.")
        print("    Objetos detectados por el Modelo, cuadros en AZUL.")
        print("\n")

    # Realiza la compración de los objetos definidos en el XML contra los detectados por el modelo
    if muestraDetalleDebug:
        print("\n+ Realiza la Comparación: ")

    resCompara = []
    auxlistObjsDetModelo = copy.copy(listObjsDetModelo)

    # Busca el objeto del XML en la lista de objetos detectados por el Modelo
    # (para eso considera la ubicación y tipo de clase)
    for objXML in listObjsXML:

        i = -1
        noEnc = True
        while noEnc and i < (len(auxlistObjsDetModelo)-1):         
          i = i + 1
          
          # calcula la Intersection over Union (IoU) de los boxes
          objIoU = calc_IoU( objXML[2], auxlistObjsDetModelo[i][2] )

          # si el IoU es casi perfecto, considera que es el objeto
          #if objIoU > 0.90: 
          #  noEnc = False
          #else:
           #  analiza si tiene algo de superposición y es de la misma clase
          noEnc = not((objIoU > 0.1) and (objXML[1] == auxlistObjsDetModelo[i][1]))

        if noEnc:

           # Si no se encuentra objeto con misma ubicación y clase del XML
          if muestraDetalleDebug:
              print(objXML[1], " no detectado por el Modelo con misma ubicación y clase ")

          # registra que ese objeto no se encontró
          resCompara.append(  (objXML[1], -1, objXML[2], "*") )

        else:

          # Si encuentra objecto en misma ubicación y  clase del XML
          if muestraDetalleDebug:
              print(objXML[1], ": ", objIoU)

          # regista que el objeto se encontró con su IoU
          resCompara.append(  (objXML[1], objIoU, auxlistObjsDetModelo[i][2], auxlistObjsDetModelo[i][1]) )

          # saca el objeto de la lista auxiliar para que no se vuelva a usar
          auxlistObjsDetModelo.pop(i)

    # Revisa los objetos detectados que no se utilizaron en la comparación anterior
    # para incluir en la comparación  
    #  objetos en la misma ubicación pero  con distinta clase
    #   u objetos detectados que no figuran en el XML
    if len(auxlistObjsDetModelo) > 0:
      if muestraDetalleDebug:
        print("\n-Se intenta asociar con ", len(auxlistObjsDetModelo), " objetos detectados del Modelo no utilizados")

      for objDet in auxlistObjsDetModelo:

        # lo compara con los que no se puedieron detectar
        i = 0
        cont = True
        while cont and i < len(resCompara):

            # sólo procesa es un objeto del XML no encontrado en Modelo 
            if (resCompara[i][1] < 0):
                # calcula la Intersection over Union (IoU) de los boxes
                objIoU = calc_IoU( resCompara[i][2], objDet[2] )

                if objIoU >= coefIoU:
                      # si están superpuestos se considera que se detectó pero le asignó mal la clase                    
                      resCompara[i] = (resCompara[i][0], objIoU, objDet[2], objDet[1] )                   
                      cont = False

                      if muestraDetalleDebug:
                          print("--- Se rectifica objeto no detectado: ", resCompara[i])
            i = i + 1

        if cont:
          # si no se utiliza, se agrega como objeto detectado de más
          resCompara.append(  ("*", 999, objDet[2], objDet[1]) )

          if muestraDetalleDebug:
              print("--- Se agrega objeto detectado por Modelo de más: ", objDet[1])

    if muestraDetalleDebug:

        print("\n+ Resultados de la Comparación:")
        print( len(resCompara), " : ", resCompara )


    # Realiza el cálculo de las Métricas de la Imagen considerando los resultados de la comparación
    ## ---------------------------------------------------------------------------------------------
    ## Nota los Verdadero Negativo (VN) se deberían calcular considerando el resto de la imagen que no tiene 
    ## objetos, por lo que no es útil para Modelos de Object Detection y no se utiliza.
    ## ---------------------------------------------------------------------------------------------
    metricasImag = [ 0, 0, 0, 0]

    hayErrorDetectado = False
    generaImagenComparacion = (muestraDetalleComparacionEnImagen != "Ninguna")
    if generaImagenComparacion:
      # imagen auxiliar para mostrar resultados comparación
      comp_image_pil = copy.deepcopy(imageCargada.convert("RGB"))
      comp_draw = ImageDraw.Draw(comp_image_pil)
      im_width, im_height = comp_image_pil.size     

    for resObj in resCompara:

        if generaImagenComparacion:
          # agrega en vectores auxiliares para hacer la evaluación por clase
          classObjReal.append( resObj[0] )
          classObjModelo.append( resObj[3] )

        if (resObj[0] == resObj[3]) and (resObj[1] >= coefIoU):

            # Objeto de misma Clase entre XML y Modelo con IoU ≥ coefIoU -> Verdadero Positivo (VP)
            posM = posVP
            if generaImagenComparacion:            
              draw_box(comp_draw, resObj[2], 4, (0,255,0)) # cuadros Verdes

        elif resObj[1] < 0:

            # Objecto del XML no encontrado en Modelo -> Falso Negativo (FN)
            posM = posFN
            hayErrorDetectado = True
            if generaImagenComparacion:
              draw_box(comp_draw, resObj[2], 4, (255,0,0)) # cuadros Rojos

        else:

            # Objeto de misma Clase entre XML y Modelo con calc_IoU(r1, r2) < coefIoU -> Falso Positivo (FP)
            # Objeto de distinta Clase entre XML y Modelo con calc_IoU(r1, r2) ≥ coefIoU -> Falso Positivo (FP)
            # Objeto detectado por el Modelo que no aparece en el Modelo -> Falso Positivo (FP)
            posM = posFP
            hayErrorDetectado = True
            if generaImagenComparacion:            
              if resObj[1] == 999:
                draw_box(comp_draw, resObj[2], 4, (255,155,255)) # cuadros Rosa
              elif (resObj[0] != resObj[3]):
                draw_box(comp_draw, resObj[2], 4, (255,255,0)) # cuadros Amarillo
              else:
                draw_box(comp_draw, resObj[2], 4, (255,155,0)) # cuadros Naranja

        metricasImag[posM] = metricasImag[posM] + 1

    # Agrega a la métrica general
    for i in range(4):
      metricasGral[i] = metricasGral[i] + metricasImag[i]

    if generaImagenComparacion and ( (muestraDetalleComparacionEnImagen == "Todas") or ( hayErrorDetectado and (muestraDetalleComparacionEnImagen == "Solo con Error") ) ):

        imMostrar = ImPIL.fromarray(np.array(comp_image_pil), 'RGB')
        display( imMostrar )
        print(" Nota colores:")
        print("     Verdaderos Positivos (igual ubicación y misma clase): cuadros en VERDE.")
        print("     Falsos Positivos (misma ubicación y distinta clase): cuadro en  AMARILLO")
        print("     Falsos Positivos (diferencia de ubicación y misma clase): cuadros en NARANJA")
        print("     Falsos Positivos (detectados de más por el Modelo): cuadros en ROSA ")        
        print("     Falsos Negativos (no detectado por el Modelo): cuadros en ROJO.")

    if ( (muestraDetalleMetricasPorImagen == "Todas") or ( hayErrorDetectado and (muestraDetalleMetricasPorImagen == "Solo con Error") ) ):
        print("\n= Matriz de Confusión para la Imagen:")        
        print("                  Modelo ")
        print(" XML   :       +          -   ")
        print("  +    :     %3d        %3d  " % (metricasImag[posVP], metricasImag[posFN]) )
        print("  -    :     %3d        %3d  " % (metricasImag[posFP], metricasImag[posVN]) )

        # Cálculo de la Exactitud
        total = (metricasImag[posVP] + metricasImag[posFP] + metricasImag[posVN] + metricasImag[posFN])
        if total>0:
            print("= Exactitud: ", round(100*(metricasImag[posVP] + metricasImag[posVN])/total, 3))

        # Cálculo de la Precisión
        total = (metricasImag[posVP] + metricasImag[posFP])
        if total>0:
            print("= Precisión: ",  round(100*metricasImag[posVP]/total,3))

        # Cálculo de la Recuperación
        total = (metricasImag[posVP] + metricasImag[posFN])
        if total>0:
            print("= Recuperación: ", round(100*metricasImag[posVP]/total,3))
    
    else:

        if hayErrorDetectado: 
            print(" -- se detecta al menos un error!")
        else:
            print(" ++ sin error detectado.")
        
# Muestra las Métricas Generales
print("\n\n===========================================================================================================")

print("\n= Matriz de Confusión General del Modelo Entrenado:")        
print("                  Modelo ")
print(" XML   :       +          -   ")
print("  +    :     %3d        %3d  " % (metricasGral[posVP], metricasGral[posFN]) )
print("  -    :     %3d        %3d  " % (metricasGral[posFP], metricasGral[posVN]) )

# Cálculo de la Exactitud
total = (metricasGral[posVP] + metricasGral[posFP] + metricasGral[posVN] + metricasGral[posFN])
if total>0:
    print("= Exactitud: ", round(100*(metricasGral[posVP] + metricasGral[posVN])/total, 3))

# Cálculo de la Precisión
total = (metricasGral[posVP] + metricasGral[posFP])
if total>0:
    print("= Precisión: ",  round(100*metricasGral[posVP]/total,3))

# Cálculo de la Recuperación
total = (metricasGral[posVP] + metricasGral[posFN])
if total>0:
    print("= Recuperación: ", round(100*metricasGral[posVP]/total,3))

print("\n===========================================================================================================\n")

# muestra la evaluación por clase
if muestraDetalleMetricasPorClaseObjeto:

    # muestra reporte de clasificación
    print("\n Reporte de Clasificación por Clase del Modelo Entrenado: ")
    print(classification_report(y_true=classObjReal, y_pred=classObjModelo))


    pd.set_option("display.max_rows", None, "display.max_columns", None)

    # muestra matriz de confusion
    print('\nMatriz de Confusión por Clase del Modelo Entrenado: ')
    CLASSES = list(set(classObjReal + classObjModelo))    
    cm = confusion_matrix(y_true=classObjReal, y_pred=classObjModelo, labels=CLASSES)
    cmtx = pd.DataFrame(
        cm, 
        index=['r:{:}'.format(x) for x in CLASSES], 
        columns=['m:{:}'.format(x) for x in CLASSES]
      )
    print(cmtx)

    print("\n===========================================================================================================\n")
